#### Homework #4

Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

что надо сделать
1. объединить в одну выборку (это только для твитов)
2. провести исследование и выявить тематики о которых говорят в твитах
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [1]:
import pandas as pd
import numpy as np
import re
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

In [2]:
from gensim import corpora
from gensim.models import *

In [3]:
morpher = MorphAnalyzer()
stopwords_list = set(get_stop_words("ru"))

In [4]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive._append(negative).sample(frac=1)
df.head()

,text,label
73580,Оказуется я обруч крутить то и не могу :( Кто ...,negative
88755,"Я сказала Кириллу,что уезжаю завтра..он позвал...",negative
42631,"@Umena что бы она там не говорила, в голову тр...",positive
100688,"Понятия не имею,почему опять начались эти стра...",negative
32283,"RT @AnyaShengel: Ахах, училка сказала, что дев...",negative


In [5]:
def lemmatize(words, lemmer = morpher, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] 
    return [w for w in lemmas if not w in stopwords and w.isalpha()] 

In [6]:
#Чистим от нецензурных слов и слов, не несущих особой смысловой нагрузки
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

In [7]:
# Формирование токенов длиной более 3 букв
token_len = 3
words_regex = re.compile(r'\w+')

def find_words(text, regex = words_regex):
    tokens = regex.findall(text)
    return [w for w in tokens if w.isalpha() and len(w) > token_len]

In [8]:
def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

In [9]:
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
73580,"[оказоваться, обруч]",negative
88755,"[кирилл, завтра, артём, маленький, милашка]",negative
42631,"[голова, транслироваться, малдёр]",positive
100688,"[понятие, начаться, страшный, резкий, головн, ...",negative
32283,"[ахи, училка, девочка, талия, месяц, пфффф]",negative


In [10]:
# Словарь уникальных токенов
dictionary = corpora.Dictionary(df['text'])

In [11]:
# Выкидывание редких слов
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')
len(dictionary.keys())

8100

In [12]:
# Векторизация твитов
corpus = [dictionary.doc2bow(text) for text in df['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [13]:
# Обработка словаря
mm = corpora.MmCorpus('tweet.model')
print(df['text'].values[0])
print(mm[0])

['оказоваться', 'обруч']
[(0, 1.0)]


In [14]:
# Обучение модели
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

In [15]:
# Определяем тематики
lda.show_topics(num_topics=3, num_words=20, formatted=True)

[(0,
  '0.011*"работа" + 0.010*"ночь" + 0.009*"дело" + 0.008*"вчера" + 0.008*"голова" + 0.008*"жаль" + 0.008*"вроде" + 0.007*"правда" + 0.006*"твиттер" + 0.006*"рука" + 0.006*"сразу" + 0.006*"место" + 0.006*"надеяться" + 0.006*"таки" + 0.006*"выйти" + 0.006*"конец" + 0.006*"ранний" + 0.006*"ахахи" + 0.005*"норма" + 0.005*"столько"'),
 (1,
  '0.024*"новый" + 0.018*"больш" + 0.014*"настроение" + 0.012*"любимый" + 0.009*"скоро" + 0.008*"фильм" + 0.007*"нужный" + 0.007*"телефон" + 0.007*"найти" + 0.007*"улица" + 0.006*"твит" + 0.006*"самый" + 0.006*"снег" + 0.006*"подарок" + 0.006*"новогодний" + 0.005*"деньга" + 0.005*"класс" + 0.005*"бояться" + 0.005*"ёлка" + 0.005*"момент"'),
 (2,
  '0.029*"хороший" + 0.022*"завтра" + 0.021*"дом" + 0.013*"утро" + 0.013*"школа" + 0.013*"друг" + 0.011*"идти" + 0.010*"прийти" + 0.010*"никто" + 0.010*"добрый" + 0.009*"неделя" + 0.008*"час" + 0.008*"нравиться" + 0.008*"самый" + 0.008*"остаться" + 0.008*"вечер" + 0.008*"урок" + 0.007*"последний" + 0.007*"пара"

In [16]:
# Оценка качества модели
print(lda.log_perplexity(corpus))

-8.029020209022738


In [17]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.00032586728188056146


In [18]:
import pyLDAvis.gensim_models as gensimvis

In [19]:
v_data = gensimvis.prepare(lda, corpus, dictionary)

In [20]:
import pyLDAvis

In [21]:
pyLDAvis.display(v_data)

Три самые популярные темы:

1) Часть жизни, проходящая в работе или учебе. Т.е. трудовые будни.

2) Тема взаимоотношений между людьми.

3) Часть жизни не связанная с работой. Отдых и развлечения.